# Metody probabilistyczne

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Ryzyko kredytowe

Każda instytucja udzielająca kredytu musi szacować zdolność kredytową i prawdopodobieństwo, czy dana osoba spłaci kredyt. W praktyce większość osób płaci prawidłowo, więc zamiast tego przewiduje się prawdopodobieństwo, że ktoś będzie miał opóźnienia w płatności w określonym czasie, tzw. **probability of default**. Jak sama nazwa wskazuje, nie jest to klasyfikacja binarna, tylko faktyczne prawdopodobieństwo, wykorzystywane przy ocenie ryzyka.

Wykorzystamy zbiór z konkursu [Give Me Some Credit](https://www.kaggle.com/c/GiveMeSomeCredit/overview). Jego zmienne to ([źródło](https://www.listendata.com/2019/08/datasets-for-credit-risk-modeling.html)):
- `SeriousDlqin2yrs` - brak spłaty raty kredytu przez 90 dni lub dłużej w ciągu ostatnich 2 lat; są to nasze klasy `y`
- `RevolvingUtilizationOfUnsecuredLines` - aktualne zapożyczenie na kartach kredytowych i pożyczkach w stosunku do maksymalnego możliwego zapożyczenia
- `age` - wiek w latach
- `DebtRatio` - aktualne całkowite zapożyczenie względem miesięcznego dochodu
- `MonthlyIncome` - miesięczne dochody
- `NumberOfOpenCreditLinesAndLoans` - liczba aktualnych pożyczek i kart kredytowych
- `NumberOfTimes90DaysLate` - ile razy w przeszłości wystąpiło opóźnienie 90 dni lub dłużej w spłacie raty kredytu
- `NumberRealEstateLoansOrLines` - liczba pożyczek z nieruchomością jako zabezpieczeniem
- `NumberOfDependents` - liczba pozostałych osób w gospodarstwie domowym
- `NumberOfTime30-59DaysPastDueNotWorse` - ile razy w ciągu ostatnich 2 lat nastąpiło opóźnienie w spłacie raty między 30 a 59 dni
- `NumberOfTime60-89DaysPastDueNotWorse` - jak wyżej, ale 60-89 dni

Ze względu na potrzebę precyzyjnych prawdopodobieństw oraz rozmiar danych (banki mają ogromną liczbę klientów), regresja logistyczna to idealny wybór dla tego zadania. Metryką w takich przypadkach jest AUROC (Area Under Receiver Operating Characteristic curve), bo bierze ona pod uwagę prawdopodobieństwo klasy pozytywnej.

Na początek wczytamy i przeanalizujemy wstępnie nasze dane. Jako zbiór testowy wykorzystamy 20% danych. Jako że mamy do czynienia z klasyfikacją niezbalansowaną, to wykorzystamy **podział ze stratyfikacją (stratified split)**, w którym proporcje klasy pozytywnej i negatywnej zostają zachowane w zbiorze treningowym i testowym.

In [ ]:
from sklearn.model_selection import train_test_split


df = pd.read_parquet("give_me_some_credit_data.parquet")
y = df.pop("SeriousDlqin2yrs")
X = df

y.value_counts(normalize=True).plot.bar(
    title="Class distribution",
    rot=0,
    xlabel="Class (defaulted or not)",
    ylabel="Class percentage",
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=0
)

Jako że naszych zmiennych nie jest zbyt wiele, możemy łatwo zweryfikować statystyki naszych danych z pomocą metod `.info()` oraz `.describe()`:

In [ ]:
df.info()

Mamy wartości brakujące w `MonthlyIncome` oraz `NumberOfDependents`. Tutaj z ekonomii wiadomo, że imputacja medianą będzie bardzo dobrym wyborem.

In [ ]:
df.describe()

W takiej tabeli warto patrzeć na średnią, odchylenie standardowe i maksimum dla każdej cechy. Mamy tutaj ewidentne outliery! Przykładowo, dla `DebtRatio`:
- średnio 353
- odchylenie standardowe 2037
- maksimum >300 tysięcy

Trzeba to będzie naprawić. Dodatkowo, nawet abstrahując od tych outlierów, patrząc na statystyki można podejrzewać dość mocno skrzywione rozkłady (co jest dość normalne w ekonomii). W takich sytuacjach lepiej używać tzw. robust statistics, na przykład mediany zamiast średniej, czy też IQR zamiast odchyleń standardowych.

**Zadanie 1 (1.5 punktu)**

1. Dokonaj imputacji danych medianą (`SimpleImputer`) oraz standaryzacji (`StandardScaler`).
2. Wytrenuj regresję logistyczną z domyślnymi hiperparametrami (`LogisticRegression`). Pamiętaj o `n_jobs=-1` (mamy dużo danych) i ustawieniu `random_state=0`. Zmierz AUROC na zbiorze testowym.
3. Dodaj usuwanie outlierów pomiędzy imputacją a standaryzacją. Skorzystaj z biblioteki feature-engine oraz klasy `OutlierTrimmer` (domyślne parametry). Wytrenuj na nowo model i porówaj wyniki.
4. Dodaj `Winsorizer` pomiędzy usuwaniem outlierów a standaryzacją, który przytnie wartości zmiennych do zakresu między średnią a 3 odchyleniami standardowymi z góry (po usunięciu outlierów powinny mieć sensowne wartości). Ponownie wytrenuj i przetestuj model.

W punkcie 3 filtrujemy także wiersze ze zbioru testowego. Niektórzy uznaliby to za niedopuszczalne, ale warto zwrócić uwagę, że usuwamy tutaj outliery uznawane za błędy, więc z drugiej strony testowanie na nich też daje błędny obraz sytuacji. Można więc to uznać za dopuszczalne, jeżeli dobrze wiemy, co robimy, i estymujemy parametry do usuwania tylko na zbiorze treningowym.

Przyda się używać innych nazw zmiennych niż `X_train` i `X_test`, żeby nie nadpisać sobie danych w kolejnych krokach.

**Uwaga:** upewnij się, że prawidłowo liczysz AUROC, przekazując do niego prawdopodobieństwa klasy pozytywnej!

Teraz mamy faktycznie czysty zbiór danych i wytrenowane pierwsze modele. Warto zwrócić jednak uwagę na dwa fakty:
1. Nie dokonaliśmy żadnego tuningu siły regularyzacji, na którą regresja logistyczna jest czuła
2. Mamy mocno niezbalansowane klasy, a uwzględniamy je z taką samą wagą

**Zadanie 2 (1 punkt)**

1. Dokonaj tuningu siły regularyzacji:
   - sprawdź 100 wartości `C` (odwrotność siły regularyzacji, jak w SVM)
   - wykorzystaj `LogisticRegressionCV`
   - wybierz model o najlepszej wartości AUROC (`scoring`)
   - pamiętaj o `random_state=0`
   - sprawdź wynik modelu na zbiorze testowym
2. Dodaj zbalansowane wagi klas (`class_weight`), z tuningiem siły regularyzacji jak wyżej. Przetestuj model.
3. Dla porównania, wytrenuj las losowy (`RandomForestClassifier`) z 500 drzewami i zbalansowanymi wagami klas, i sprawdź wyniki.
4. Skomentuj wyniki:
   - czy twoim zdaniem to ogólnie dobry wynik dla takiego problemu?
   - czy warto było poświęcić czas na preprocessing danych?
   - jak wypadła regresja logistyczna w porównaniu do lasu losowego? Czy jest do dla ciebie oczekiwany, czy zaskakujący wynik?

// skomentuj tutaj

Sprawdźmy jeszcze, jak dobrze nam idzie z predykcjami prawdopodobieństwa. Regresja logistyczna jest z natury dobrze skalibrowana, ale nie ma takiej gwarancji na każdym zbiorze danych. W szczególności może mieć problemy na zbiorach niezbalansowanych, przez bardzo niewielką ilość klasy pozytywnej (a priori jest rzadka).

Do rysowania krzywych kalibracji służy klasa `CalibrationDisplay`. W perfekcyjnym przypadku prawdopodobieństwa modelu rosną liniowo raz z "pewnością", że punkt należy do klasy pozytywnej. Dla zainteresowanych, [dokumentacja Scikit-learn](https://scikit-learn.org/stable/modules/calibration.html#calibration-curves) omawia te krzywe nieco dokładniej.

In [ ]:
from sklearn.calibration import CalibrationDisplay

clf = clf  # put model variable from last exercise here

CalibrationDisplay.from_estimator(clf, X_test_scaled, y_test_out_trimmed, n_bins=10)

Krótko mówiąc, nie jest dobrze! Nasz model praktycznie zawsze bardzo mocno zaniża prawdopodobieństwo. W połączeniu z wysokim AUROC oznacza to, że w klasyfikacji generalnie nieźle sobie radzi, dla rosnącego prawdopodobieństwa dość szybko przypisuje punkt do klasy pozytywnej, ale finalne wartości prawdopodobieństw są notorycznie zbyt niskie.

Do poprawienia tej sytuacji służy `CalibratedClassifierCV`, który na podstawie predykcji naszego modelu oraz zbioru treningowego dotrenowuje dodatkowy model do doprecyzowania prawdopodobieństw. Może być to albo regresja logistyczna, albo tzw. regresja izotoniczna ([isotonic regression](https://scikit-learn.org/stable/modules/isotonic.html#isotonic)) - specjalny model wykorzystujący fakt, że prawdopodobieństwa są stricte rosnące od 0 do 1.

**Zadanie 3 (1 punkt)**

Wytrenuj kalibrator prawdopodobieństwa z pomocą `CalibratedClassifierCV`. Wykorzystaj tryb regresji izotonicznej i 10-krotną walidację skrośną. Przekaż do niego wytrenowany estymator wykorzystany w poprzednim ćwiczeniu. Narysuj krzywą kalibracji na zbiorze testowym, jak w poprzednim przykładzie.

Skomentuj, jak udało się poprawić kalibrację modelu. Czy twoim zdaniem taki model przewiduje już dość dobrze prawdopodobieństwa?

// skomentuj tutaj

## Detekcja spamu

Wykorzystamy teraz klasyfikator Naiwnego Bayesa do klasyfikacji binarnej tekstów - czy email jest spamem, czy też nie (tzw. "ham"). Takie zadanie nazywa się też spam/ham classification. Zbiorem będzie [Enron Spam dataset](http://nlp.cs.aueb.gr/software_and_datasets/Enron-Spam/index.html), zebrany na potrzeby artykułu:

> Metsis, Vangelis, Ion Androutsopoulos, and Georgios Paliouras. *"Spam filtering with Naive Bayes - which Naive Bayes?."* CEAS. Vol. 17. 2006. [link](https://userweb.cs.txstate.edu/~v_m137/docs/papers/ceas2006_paper_corrected.pdf)

[Upadek firmy energetycznej Enron](https://userweb.cs.txstate.edu/~v_m137/docs/papers/ceas2006_paper_corrected.pdf) i skandal wywołany wiadomościami o nieprawidłowościach finansowych spowodowały, że podczas śledztwa i procesu zmuszono firmę do upublicznienia pełnej zawartości skrzynek mailowych 150 pracowników. Jest to doskonałe źródło prawdziwych danych, typowych emaili niebędących spamem, i było wykorzystywane wielokrotnie na potrzeby tworzenia wielu różnych zbiorów danych. Na potrzeby powyższego artykułu wybrano 6 pracowników z dużą liczbą emaili.

Maile będące spamem otrzymano z 4 różnych źródeł:
1. [Zbioru danych SpamAssassin](https://spamassassin.apache.org/old/publiccorpus/) - stworzony z pomocą [Apache SpamAssassin](https://spamassassin.apache.org/), narzędzia open source do detekcji spamu. Maile niebędące spamem są w nim bardzo specyficzne (pochodzą ze skrzynek fundacji Apache i dotyczą praktycznie tylko programownaia), za to spam jest uniwersalny.
2. [Project Honeypot](https://www.projecthoneypot.org/) - projekt mający na celu wykrywanie spammerów (w tym przez specjalne pułapki, tzw. spam baiting) i tworzenie baz do filtrowania spamu.
3. [Bazę spamu Bruce'a Guentera](http://untroubled.org/spam/) - gromadzi on cały spam, który dostaje na skrzynki (w tym takie do spam baitingu) od 1997 roku.
4. Maile ze spamem zgromadzone przez trzeciego autora artykułu, Georgiosa Paliourasa.

Artykuł prezentuje spersonalizowane kolekcje maili, dla każdego z 6 wybranych pracowników Enronu. My natomiast połączymy wszystkie dane i stworzymy prosty, uniwersalny filtr. Nasz filtr będzie operował tylko na tytułach i treści maili - realnie wdrażane systemy są dużo bardziej rozbudowane i wykorzystują metadane, bazy adresów, analizę tagów HTML i XML etc.

Wykorzystamy przetworzoną postać tego zbioru w postaci pliku CSV z interesującymi nas elementami. Skrypt do tego wzięto [z tego repozytorium na Githubie](https://github.com/MWiechmann/enron_spam_data?fbclid=IwAR1icPxic8TDD6HGk6empohwcZ76uig0NxwxK7sWbczinBNfxEEt-ETparA).

In [ ]:
df = pd.read_csv("enron_spam_data.csv")
df

Na pierwszy rzut oka widać parę problemów:
- kolumny `Message ID` i `Date` są niepotrzebne
- niektóre wiadomości mają tylko tytuł lub tylko wiadomość
- klasa jest zapisana jako string spam/ham

**Zadanie 4 (0.5 punktu)**

1. Uzupełnij wartości brakujące w kolumnach `Subject` i `Message` pustymi stringami.
2. Połącz kolumny `Subject` i `Message` w jedną kolumnę `text`, ze spacją pomiędzy nimi. Może się przydać metoda `pd.Series.str.cat()`. Wyodrębnij ją do nowej zmiennej `texts`.
3. Zamień wartość `ham` na 0 i `spam` na 1 w kolumnie `Spam/Ham`. Wyodrębnij ją do nowej zmiennej `y`.

Sprawdźmy jeszcze rozkład klas.

In [ ]:
class_percentages = y.value_counts(normalize=True)
class_percentages = round(class_percentages * 100, 2)

ax = class_percentages.plot.bar(title="Class distribution", rot=0)

ax.bar_label(ax.containers[0])

Ten zbiór w całości jest o tyle specyficzny, że dodano tutaj aż tyle spamu, że jest go nawet więcej niż zwykłych emaili. W praktyce spamu jest dużo mniej, ok. 1/3 maili. Tutaj uznajemy, że klasy są zbalansowane.

Typową metryką w klasyfikacji spamu jest **precyzja (precision)**, bo wyznacza ona, jak dobrzy jesteśmy w oznaczaniu spamu. Recall ma mniejsze znaczenie, bo bardziej zależy nam na minimalizacji FP - ważne maile nie mogą trafiać do spamu, natomiast jeżeli spam czasem trafi do zwykłej skrzynki (FN), to nie ma wielkiego znaczenia. Często używa się też F1-score jako metryki.

W przypadku tych metryk (precision/recall/F1-score) zawsze trzeba sprawdzić co najmniej dwie z nich, żeby uniknąć przypadkiem sytuacji, że mamy np. precision bliskie 100%, a dramatycznie niski recall - to też nie byłoby akceptowalne. Trzeba też uważać podczas walidacji skrośnej - np. klasyfikator przewidujący cały czas klasę pozytywną miałby 100% recall, a cały czas negatywną 100% precision, więc typowo najwygodniej jest optymalizować F1-score.

Na początek podzielimy zbiór na treningowy i testowy. Naszymi cechami będą zliczenia wystąpień poszczególnych słów, obliczone z pomocą klasy `CountVectorizer` ([dokumentacja](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)). Na start podamy `binary=True`, dzięki czemu cechy będą binarne - czy słowo wystąpiło, czy nie. Dzięki temu będziemy mogli użyć Naiwnego Bayesa z rozkładem dwumianowym - `BernoulliNB`. Wykonamy minimalny preprocessing, usuwając tylko tzw. stop words, czyli "nudne", niepotrzebne słowa, typu "he", "she", "the" etc.

Zauważ, że wejściem do metody `.fit()` jest lista tekstów, ze względu a domyślne `input="content"`. Można by tu też wskazać listę ścieżek do plików z tekstami (`input="filename"`).

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score, precision_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB


texts_train, texts_test, y_train, y_test = train_test_split(
    texts,
    y,
    test_size=0.25,
    random_state=0,
    stratify=y,
)

count_vectorizer = CountVectorizer(
    stop_words="english",
    binary=True,
)
count_vectorizer.fit(texts_train)

X_train = count_vectorizer.transform(texts_train)
X_test = count_vectorizer.transform(texts_test)

print(f"Dictionary size: {X_train.shape[1]}")

clf_nb_bernoulli = BernoulliNB(binarize=None)
clf_nb_bernoulli.fit(X_train, y_train)

y_pred = clf_nb_bernoulli.predict(X_test)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Precision: {100 * precision:.2f}%")
print(f"F1-score: {100 * f1:.2f}%")

Wynik jest prawie doskonały. Może uda się jeszcze wycisnąć ten ostatni procent?

**Zadanie 5 (2 punkty)**

1. Oblicz wystąpienia słów bez binaryzacji, otrzymując drugi zbiór cech.
2. Wytrenuj model multinomial Naive Bayes na otrzymanym zbiorze. Sprawdź precyzję i F1-score na zbiorze testowym.
3. Stwórz `Pipeline` z połączenia `CountVectorizer` i `MultinomialNB`. Podaj te opcje, co wcześniej do CountVectorizera.
4. Dokonaj tuningu hiperparametrów:
   - `ngram_range` - jakie n-gramy rozważać, sprawdź wartości `[(1, 1), (1, 2), (1, 3)]` (same słowa, + bigramy, + bigramy i trigramy)
   - `max_df` - w jakim maksymalnie ułamku tekstów w zbiorze może wystąpić słowo, sprawdź wartości `[0.5, 0.6, 0.7, 0.8, 0.9, 1.0]`
   - wybierz model o najwyższej precyzji
   - użyj standardowych ustawień: 5-krotna walidacja skrośna, `n_jobs=-1`
   - sprawdź precyzję i F1-score na zbiorze testowym dla znalezionych optymalnych hiperparametrów
5. Wypisz optymalne hiperparametry. Sprawdź, jak zmienił się rozmiar słownika przy tych hiperparametrach.
6. Skomentuj uzyskane hiperparametry. Jaki zakres (kontekst) uwzględniamy przy takich n-gramach? Czy uwzględniamy wszystkie słowa/bigramy/trigramy?

**Uwaga:**
1. Do metody `.fit()` możesz przekazać `texts_train` i `y_train`. Jako że `CountVectorizer` ma interfejs transformera (ma metodę `.transform()`), to Scikit-learn przekształci nim teksty do macierzy `X_train` i to ją przekaże dalej do klasyfikatora.
2. Przy tworzeniu pipeline'u może się przydać [ta odpowiedź na StackOverflow](https://stackoverflow.com/a/64258746/9472066).
3. Pamiętaj, że do testowania też trzeba przekazać teksty, które przekształcimy w środku pipeline'u.

// skomentuj tutaj

W przypadku tak wysokich wyników raczej nie ma sensu sprawdzać bardziej złożonych technik, typu ważenie słów (TF-IDF, BM25), upraszczanie słów (stemming, lematyzacja) etc. W przypadku bardziej złożonych tekstów mogłoby to być przydatne.

Naiwnego Bayesa do klasyfikacji tekstu używa się obecnie relatywnie rzadko. Potrafi on jednak działać równie dobrze, jak transformery, kiedy dla danego zadania mają znaczenie same wystąpienia słów czy tematyka tekstu, a nie dokładna semantyka zdań. Naiwny Bayes jest przy tym oczywiście bez porównania prostszy i szybszy.

## Dane z ekspresji genów

Naszym ostatnim zbiorem będzie [GSE14520](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE14520), opublikowany przez [SBCB Lab](https://sbcb.inf.ufrgs.br/cumida)). Jest to zbiór ekspresji genów z sekwencjonowania RNA, zebrany na potrzeby artykułu:

> Feltes, B.C.; Chandelier, E.B.; Grisci, B.I.; Dorn, M. (2019) *"CuMiDa: An Extensively Curated Microarray Database for Benchmarking and Testing of Machine Learning Approaches in Cancer Research"*. Journal of Computational Biology, 26 (4), 376-386. [link](https://sci-hub.se/10.1089/cmb.2018.0238)

Dane o ekspresji DNA pozwalają na analizę genetyczną i związku pomiędzy genomem pacjenta a podatnością na choroby. Jest to szczególnie istotne w kontekście nowotworów, gdyż wiele ich typów powstaje przez błędy właśnie w procesie ekspresji genów. Mikromacierze (microarrays) służą do przetwarzania i analizy danych dużej skali w genetyce i dostarczają danych o ogromnej liczbie danych pacjenta. Oczywiście samo gromadzenie próbek, gdzie każdą jest osobny pacjent, jest bardzo drogie. Daje to zbiory danych o bardzo małej liczbie próbek, nawet zaledwie kilkudziesięciu czy kilkuset, a za to dziesiątki tysięcy cech.

Zbiór CuMiDa zawiera 78 zbiorów danych, wybranych z bazy GEO (Gene Expression Omnibus) i poddanych preprocessingowi, np. normalizacji danych pomiędzy płytkami pomiarowymi różnych typów. Wynikowe zadania dotyczą klasyfikacji czy detekcji nowotworu na podstawie analizy genomu pacjenta. Cechy w zbiorach danych dotyczą siły ekspresji poszczególnych genów, a konkretnie znormalizowanej wartości - czy gen jest słabiej, czy silniej wykazywany u danej osoby względem grupy kontrolnej. Są one na etapie preprocessingu standaryzowane do z-score (odjęcie średniej, podzielenie przez odchylenie standardowe), co skutkuje danymi o podobnej skali, bardziej zbliżonym rozkładzie, i pozwala porównywać ze sobą dane poszczególnych pacjentów.

Użyjemy zbioru GSE 14520, który jest największym z kolekcji CuMiDa i zawiera 357 próbek oraz 22278 genów. Dotyczy klasyfikacji tkanek nowotoworowych (klasa pozytywna) vs zdrowych (klasa negatywna) dla raka wątrobowokomórkowowego wątroby (hepatocellular carcinoma, HCC). Jest to często występujący, a przy tym bardzo złośliwy nowotwór (trzeci najbardziej śmiertelny), będący często skutkiem innych chorób wątroby.

Na start wczytamy dane, sprawdzimy rozkład klas, oraz zwizualizujemy rozkłady kilku przykładowych cech.

In [ ]:
df = pd.read_csv("Liver_GSE14520_U133A.csv")

y = df["type"].map({"normal": 0, "HCC": 1})
df = df.drop(columns=["samples", "type"])

fig, ax = plt.subplots(1, 5, figsize=(12, 2))
for i in range(5):
    df.hist(column=df.columns[i], ax=ax[i])

plt.show()

pd.Series(y).value_counts().plot.bar(title="Class distribution", rot=0, figsize=(3, 3))

**Zadanie 6 (1 punkt)**

Ze względu na niewielki rozmiar takich zbiorów, walidacji skrośnej używa się **do testowania**. Podział na zbiór treningowy i testowany jest wykonywany na kolejnych foldach. Dla ekstremalnie małych zbiorów można też wykorzystać **Leave-One-Out Cross-Validation (LOOCV)**, gdzie zbiór testowy to jedna próbka naraz. W razie potrzeby tuningu hiperparametrów można też wykonać wewnętrzną walidację skrośną na aktualnym foldzie treningowym - technikę tę nazywamy **nested cross-validation (nested CV)**.

1. Dokonaj podziału na zbiór treningowy i testowy z pomocą 3-fold CV (klasa [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)), analogicznie do oryginalnego artykułu. Wykorzystaj losową kolejność (`shuffle=True`) oraz stały `random_state=0`.
2. Wytrenuj klasyfikator [GaussianNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html).
3. Dokonaj walidacji na zbiorach testowych, wykorzystując celność (accuracy).
4. Wypisz wartość średnią (w procentach) celności na danych testowych, oraz odchylenie standardowe, oba zaokrąglone do 2 miejsca po przecinku.
5. Zmierz czas dla całości procesu, np. z pomocą modułu `time` lub biblioteki `tqdm`.
6. Porównaj wyniki z oficjalnymi podanymi [na stronie SBCB Lab](https://sbcb.inf.ufrgs.br/cumida) dla tego zbioru danych - jak dobrze radzi sobie klasyfikator Naiwnego Bayesa?

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB


# accuracy scores over folds
scores = []

# splitter object, e.g. KFold
...

# split in loop
for train_idxs, test_idxs in ...:
    # use train and text indexes
    ...

    # train, evaluate, append score
    ...


# calculate mean and standard deviation test score
...

// skomentuj tutaj

Skoro mamy dobrze działający klasyfikator, to warto dobrze zrozumieć, jak działa pod spodem. Jest to też doskonała okazja, żeby nauczyć się, jak tworzyć własne klasyfikatory zgodne z API scikit-learn. Robi się w skrócie to tak ([dokładny tutorial](https://scikit-learn.org/stable/developers/develop.html)):
1. Tworzymy klasę dziedziczącą po `BaseEstimator`, `ClassifierMixin` oraz ewentualnie innych klasach, które nas interesują.
2. Wszystkie hiperparametry są argumentami konstruktora. Każdy jest przypisywany, bez żadnych zmian, do atrybutu o dokładnie tej samej nazwie.
3. Implementujemy metody `.fit(X, y)`,`.predict(X)`, oraz ewentualnie `.predict_proba(X)`.
4. Metoda `.fit()` musi zwracać `self`, a estymowane parametry przypisuje jako atrybuty (tworzone w `.fit()`) z underscore na końcu, np. `.weights_`. Często wywołuje się `np.asarray()` na `X` i `y`, żeby upewnić się, że będą to macierze NumPy'a.
5. Na początku metody `.fit(X, y)` używamy funkcji `check_X_y()`, żeby dokonać podstawowej walidacji wejścia. Na początku metody `.predict(X)` analogicznie używamy funkcji `check_array()` na wejściu `X`.

Taki interfejs gwarantuje, że nasz własny model będzie mógł być używany np. w `GridSearchCV`.

**Zadanie 7 (3 punkty)**

Uzupełnij implementację własnej klasy `CustomGaussianNB`, która będzie realizować Naiwnego Bayesa dla rozkładu normalnego.

1. Zakładamy, że klasy są liczbami całkowitymi, z wartościami `[0, 1, ..., liczba klas - 1]`.
2. Rozkład a priori (prior) dla każdej klasy to proporcja tej klasy w zbiorze treningowym. Naive Bayes używa log-sum-exp trick, więc można od razu obliczyć logarytm i zapisać log-prior.
3. Gaussian NB zakłada, że każda cecha w obrębie klasy ma rozkład normalny. Opisują go dwa parametry: średnia i wariancja. Będziemy więc mieli `2*K` parametrów dla `K` klas. Trzeba je wyestymować w metodzie `.fit()` i zapisać jako macierze parametrów, np. `.means_` i `.variances_`.
4. Do wariancji klas dodaj niewielką stałą, żeby uniknąć problemów numerycznych, np. 1e-9. Scikit-learn nazywa to `var_smoothing`.
5. Podczas predykcji trzeba wybrać klasę o najwyższej wiarygodności. Dla każdego z przykładów obliczamy log-likelihood dla każdej klasy, dodajemy log-prior klasy, i w ten sposób otrzymujemy wartość rozkładu a posteriori (posterior). Zgodnie z zasadą maximum a posteriori (MAP), wybieramy klasę o najwyższej z tych wartości.
6. Pamiętaj, żeby użyć prawidłowo metod gwarantujących zgodność ze scikit-learn, jak opisano powyżej.
7. Przetestuj klasę na przykładzie zbioru użytego w zadaniu 6. Wynik powinien być identyczny z tym ze scikit-learn.

Dla uproszczenia, nie będziemy się tutaj zajmować metodą `.predict_proba()`. Jej implementacja to stosunkowo niewielka adaptacja metody `.predict()`.

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils import check_array, check_X_y


class CustomGaussianNB(BaseEstimator, ClassifierMixin):
    def fit(self, X, y):
        # check arrays
        ...

        # make sure that y has values from 0 to num_classes - 1
        ...

        # vector of possible classes
        self.classes_ = ...

        # parameters:
        # - priors, i.e. proportions of classes in the training set
        # - mean of each feature in each class
        # - variance of each feature in each class
        self.log_priors_ = ...
        self.means_ = ...
        self.variances_ = ...

        for class_idx in self.classes_:
            # select X and y from the current class
            ...

            # calculate log-prior, i.e. logarithm of current class proportion
            ...

            # calculate means and variances of current class
            ...

        return self

    def predict(self, X):
        preds = [
            self._single_sample_predict(x) for x in X
        ]
        return np.array(preds)
    
    def _single_sample_predict(self, x):
        class_scores = np.empty(len(self.classes_))
        for class_idx in self.classes_:
            log_prior = ...
            log_likelihood = ...

            class_scores[class_idx] = ...

        return ...


In [ ]:
# test your class!

// skomentuj tutaj

## Zadanie dodatkowe (3 punkty)

Zamiast zakładać konkretny rozkład prawdopodobieństwa dla zmiennych w Naiwnym Bayesie, możemy wprost estymować go z danych treningowych. Takie nieparametryczne podejście z estymacją rozkładu z danych nazywamy kernel density estimation (KDE), a klasyfikator KDE Naive Bayes. Został on opisany na slajdach bonusowych na prezentacji z zajęć.

1. Zaimplementuj KDE Naive Bayes. Dla każdej zmiennej wytrenuj instancję [KernelDensity](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KernelDensity.html). Podczas predykcji użyj metody `.score_samples()`, która zwróci log-likelihoods próbek. Obliczanie log-prior nie ulega zmianie, jedyna zmiana to sposób obliczania log-likelihoods.
2. W metodzie `__init__()` klasa przyjmuje argument `bandwidth` z wartością domyślną 1.0, który będzie parametrem estymatora KDE. Wyznacza on szerokość kernela przy estymacji rozkładu, czyli jego "gładkość". Wąskie kernele mają większą wariancję, ale też są bardziej elastyczne.
3. Wytrenuj KDE Naive Bayes na wybranych 2-3 zbiorach z kolekcji CuMiDa. Porównaj wyniki z Gaussian NB. Dokonaj tuningu parametru `bandwidth`, zazwyczaj jego zmniejszenie bywa pomocne.
4. Skomentuj wyniki.


In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neighbors import KernelDensity
from sklearn.utils import check_array, check_X_y


class KernelDensityEstimationNB(BaseEstimator, ClassifierMixin):
    """
    Naive Bayes with KDE estimation of feature distributions.

    Parameters
    ----------
    bandwidth : float
        Kernel bandwidth
    """

    def __init__(self, bandwidth: float = 1.0):
        self.bandwidth = bandwidth

    def fit(self, X, y):
        ...

    def predict(self, X):
        ...
